In [1]:
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, GPT2TokenizerFast
from transformers import pipeline
import torch
from torch import nn
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import csv
import re
import matplotlib.pyplot as plt
from datasets import load_dataset, Dataset, load_metric
from sklearn.model_selection import train_test_split
%matplotlib inline

In [10]:
data = pd.read_csv('/nas/home/gujiashe/trans/yago310_top10_predictions_val.tsv', sep="\t", header=0, index_col=0)
perplexities = []
cnt = 0
text = []
label = []
flag = True
for head, relation, pred_tail, true_tail, rank, score in tqdm(data.to_numpy()):
    if rank>10:
        continue
    if cnt%10 == 0:
        flag = True
    head = " ".join(head.split("_"))
    relation = re.sub( r"([A-Z])", r" \1", relation).lower()
    # print(relation)
    pred_tail = " ".join(pred_tail.split("_"))
    true_tail = " ".join(true_tail.split("_"))
    sentence = " ".join([head, relation, pred_tail])
    if pred_tail == true_tail:
        label.append(1)
        text.append(sentence)

    elif flag:
        label.append(0)
        text.append(sentence)
        flag = False
    # text.append(sentence)
    # if pred_tail == true_tail:
    #     label.append(1)
    # else:
    #     label.append(0)
    # break
    cnt+=1

    # if cnt>10:
    #     break



100%|██████████| 49780/49780 [00:00<00:00, 143338.40it/s]


In [3]:
# df_data = pd.DataFrame()
# df_data["text"] = text
# df_data["label"] = label
# df_data["text"].to_csv("lp.data.txt", header=None, index = None)
# df_data["label"].to_csv("lp.gold.txt", header=None, index = None)

In [11]:
df_data = pd.DataFrame()
df_data["text"] = text
df_data["label"] = label

train, test = train_test_split(df_data, test_size=0.2, random_state=42, stratify=df_data["label"])


train["text"].to_csv("/nas/home/gujiashe/kb/experiments/train.data.txt", header=None, index = None)
train["label"].to_csv("/nas/home/gujiashe/kb/experiments/train.gold.txt", header=None, index = None)
test["text"].to_csv("/nas/home/gujiashe/kb/experiments/dev.data.txt", header=None, index = None)
test["label"].to_csv("/nas/home/gujiashe/kb/experiments/dev.gold.txt", header=None, index = None)

In [12]:
# pred = pd.read_csv('/nas/home/gujiashe/critic/dev1.pred.txt', header=None)
gold = pd.read_csv('/nas/home/gujiashe/kb/experiments/train.gold.txt', header=None)
gold.value_counts()

1    2489
0    2488
dtype: int64

In [5]:
(pred==gold).value_counts()

True     642
False    602
dtype: int64

In [14]:
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
# from transformers import AutoTokenizer
# from transformers import DataCollatorWithPadding
# dataset = Dataset.from_pandas(df_data)
# dataset = dataset.train_test_split(test_size=0.2)
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)
# tokenized_dataset = dataset.map(preprocess_function, batched=True)

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [101]:

# from transformers import Trainer


# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
        
#         log_pt = torch.log_softmax(logits, dim=1)
#         pt = torch.exp(log_pt)
#         log_pt = (1 - pt) ** 2 * log_pt
#         labels = labels.to("cuda")
#         # log_pt.to("cuda")
#         loss = torch.nn.functional.nll_loss(log_pt, labels, None, reduction='mean', ignore_index=-100)

#         return (loss, outputs) if return_outputs else loss

In [15]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=5,
#     weight_decay=0.01,
# )
# metric = load_metric('accuracy')

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics = compute_metrics
# )

# trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4976
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 390
/nas/home/gujiashe/miniconda3/envs/transformers/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=390, training_loss=0.6137779040214343, metrics={'train_runtime': 167.8138, 'train_samples_per_second': 148.26, 'train_steps_per_second': 2.324, 'total_flos': 147485889978624.0, 'train_loss': 0.6137779040214343, 'epoch': 5.0})

In [16]:
# trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1244
  Batch size = 64


{'eval_loss': 0.6657101511955261,
 'eval_accuracy': 0.5908360128617364,
 'eval_runtime': 4.125,
 'eval_samples_per_second': 301.574,
 'eval_steps_per_second': 4.848,
 'epoch': 5.0}

In [104]:
# input_ids = tokenizer("Diane_Varsi diedIn Los_Angeles", return_tensors="pt").input_ids.to("cuda")
# # _model = wrap
# model(input_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.1031, -2.5563]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)